<a href="https://colab.research.google.com/github/oscarlopezbello/Parsing_hv/blob/main/Demo_Parseador_HV_PeriferiaIT_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DEMO PARSEADOR HOJAS DE VIDA

En este demo se busca realizar una demostracíon de como se puede extraer la información de una hoja de vida contando con unos parametros de entrada en formato json. la hoja de vida de referencia esta en el siguiente link [Hoja de Vida Referencia](https://drive.google.com/file/d/1l0P5Wgsay-e8s7HQ_eNi6as94hDf4k5a/view?usp=sharing). Este codigo se encarga de recibir estos parametros de entrada en formato json como lo pueden ser: 
1. sueldo 
2. conocimientos en lenguajes o tecnologias especificas.
3. Idiomas que domine el candidato.
4. Si cuenta o no con tarjeta profesional registrada en su hoja de vida.
5. Estudios especificos en algun campo que se requiera.

La funcion parsing_hv se encarga de procesar estos requisitos y devolver un json nuevo con los campos que cumpla la personas, adicionalmente de entregar otros datos de la hoja de vida que para este demo son:

1. Nombre del Candidato.
2. Experiencia con la que cuenta en la hoja de vida.
3. edad del candidato.
4. Si cuenta o no con tarjeta profesional registrada en su hoja de vida.
5. Sueldo o aspiracion Salarial registrada en la hoja de vida del candidato.
6. Conocimientos esecificos con los que cuente la persona en: Bases de Datos, Lengiajes de Programación, frameworks, herramientas de Nube. Estos campos se retornaran vacios en caso que el candidato no cuente con alguna de estas habilidades o conocimientos.
7. Idiomas que maneje y registre en su hoja de vida segun requerimiento de entrada. 
8. Estudios que maneje y registre en su hoja de vida segun requerimiento de entrada. 
9. Una ponderación en porcentaje de coincidencia de los valores solcitidados contra los valores o requisitos encontrados en la hoja de vida, esto expresado en un procentaje sobre un 100%.

A continuacion se Explica un paso a paso de como funciona este codigo.

In [13]:
import json
import nltk
import re
import os
from datetime import datetime
import base64
from pathlib import Path
from google.colab import drive

In [14]:
## Se realiza importación de libreria para poder importar el fichero desde ubicación drive para efectos del demo.
#drive.mount('/content/drive')

In [15]:
def parsing_hv (json_frontend):

    def nombre_candidato (cadena):
        tags = nltk.pos_tag(nltk.word_tokenize(cadena))
        nouns = [(word, tag) for word, tag in tags if tag in ['NN','NNS', 'NNP', 'NNPS']]
        nombres = nouns[0:3]
        nombre = ""
        for x in nombres:
            a="".join(x)
            nombre = nombre+a+""
            nombre = nombre.replace("NNP", " ").replace("\uf095","").replace("Ingeniería", "").replace("NNS"," ").replace("\uf00c","").replace("NN"," ")
        return nombre

    def read_file(path):
        items = []
        f = open(path)
        for x in f:
            items.append(str(x.replace('\n','')))
        f.close()
        return items

    def habilidades (habilidad, string_json, texto):
        lista = []
        for a in habilidad:
            a = a.lower()
            a = a.strip()
            if a in texto:
                lista.append(a)
        json_hv[string_json] = lista
            
    def coincidencias(busqueda, json_hv):
        set_parametros = set()
        total_encontrado = 0
        set_valores = set()
        dicc_parametros = {}
        for key in busqueda.keys():
            set_valores.add(key)
        
        for key, value in busqueda.items():
            lista=[]
            if type(value) == list:
                for i in value:
                    if i == "No aplica":
                        set_valores.remove(key)
                    lista.append(i.lower())
                        
                if len(value) == 0:
                    set_valores.remove(key)
                elif key in set_valores:
                    dicc_parametros[key] = len(value)
                busqueda[key]=lista   
                
        ponderacion = 100 / len(set_valores)

        for key, value in json_hv.items():
            contador = 0
            if key in set_valores:
            
                for i in value:
                    if str(i).lower() in busqueda[key]:
                        contador += 1
                dicc_parametros[key] = (dicc_parametros[key], contador)

        coincidencia = 0
        for key, value in dicc_parametros.items():
            coincidencia += (value[1] / value[0]) * ponderacion    
        return coincidencia

    def hv_base64 (ruta):
        data = open(ruta, "rb").read()
        encoded = base64.b64encode(data)
        hv_base64 = encoded.decode('utf-8')
        return hv_base64
        
    parametros = json_frontend
    
    bases_datos = parametros["bases_de_datos"]
    lenguajes_programacion = parametros["lenguajes_de_programacion"] 
    frameworks = parametros["frameworks"]
    servicios_nube = parametros["nube"]
    cargos = parametros["cargos"]
    idiomas = parametros["idiomas"]
    estudios = parametros["estudios"]
    tarjeta = parametros["tarjeta_profesional"]
    sueldo = parametros["sueldo"]

    files = []
    for file in os.listdir("https://github.com/oscarlopezbello/Parsing_hv.git"):
        if file.endswith(".txt"):
            files.append("https://github.com/oscarlopezbello/Parsing_hv.git" + "/" + file)

    lista_json = []
    for file in files:
        documento = open(file,encoding='utf-8')
        documento_txt = str(documento.read())
        json_hv = {}
        texto_limpio = ""
        
        filename = Path(file).stem
        ruta_pdf= "https://github.com/oscarlopezbello/Parsing_hv.git"
        link_hv = f"{ruta_pdf}/{filename}.pdf"
        hv_encoded = hv_base64(link_hv)
        #json_hv["hv_base64"] = hv_encoded

        for b in documento_txt:
            b = b.replace("\uf00c"," ").replace("\uf041"," ").replace("\uf003"," ").replace("\uf095"," ").replace("\uf19c"," ")
            b = b.rstrip()
            b = b.lstrip()
            b = b.strip()
            texto_limpio = texto_limpio + b
        nombre = nombre_candidato(documento_txt)
        json_hv["nombre"] = nombre
        print(json_hv["nombre"])
        texto_limpio_base = texto_limpio.lower()

        exp_temp=re.findall("[a-zA-Z0-9_]* [0-9]{4} - [a-zA-Z0-9_]* [0-9]{4}",documento_txt.lower() )
        fechas_experiencias=[]
        for a in exp_temp:
            a = a.replace("enero","01").replace("febrero","02").replace("marzo","03").replace("abril","04").replace("mayo","05").replace("junio","06").replace("julio","07").replace("agosto","08").replace("septiembre","09").replace("octubre","10").replace("noviembre","11").replace("diciembre","12")
            a = a.split(" - ")
            fechas_experiencias = fechas_experiencias + a
        fechas_experiencias1=[]
        for lista in fechas_experiencias:    
            lista= lista.replace(" ","/01/").replace("/20", "/").replace("/199", "/9").replace("/198","/8")
            fechas_experiencias1 = fechas_experiencias1 + [lista]
        try:

            if len(fechas_experiencias1)>3:
                strDate = fechas_experiencias1[-2]
                date2 = fechas_experiencias1[0]
                objDate = datetime.strptime(strDate, '%m/%d/%y')
                objDate1 = datetime.strptime(date2, '%m/%d/%y')
                total = (objDate1 - objDate)/365
                experiencia_total = total.days
                json_hv["experiencia"] = str(experiencia_total)
            else:
                json_hv["experiencia"] = "no cuenta con tiempo de experiencias"

        except IOError as e:
            json_hv["experiencia"] = "no cuenta con tiempo de experiencias"


        edad=re.findall("[|] [0-9]{2} años",documento_txt )
        edad = "".join(edad).replace("| ","").replace(" años","")
        json_hv["edad"] = edad
        

        for t in tarjeta:
            if t =="si":
                if "tarjeta profesional" in documento_txt:
                    json_hv["tarjeta_profesional"] = "si"
                elif "t.p" in texto_limpio_base:
                    json_hv["tarjeta_profesional"] = "si"
                else:
                    json_hv["tarjeta_profesional"] = ""
            else:
                json_hv["tarjeta_profesional"] = ""    

                    
        sueldo_hv = re.findall("[$][0-9]{1} A [$][0-9]{1}" or "[$][0-9]{1}[,][0-9]{1} A [$][0-9]{1}[,][0-9]{1}",documento_txt ) 
        if len(sueldo_hv)>0:
            sueldo_hv = "".join(sueldo_hv).replace("$","")
            sueldo_hv = sueldo_hv[-1]
            for i in sueldo:
                if i<sueldo_hv:
                    json_hv["sueldo"] = ""
                else:
                    json_hv["sueldo"] = sueldo_hv 
        else:
            json_hv["sueldo"] = "no cuenta con aspiracion salarial"

        habilidades(bases_datos,"bases_de_datos",texto_limpio_base)
        habilidades(lenguajes_programacion,"lenguajes_de_programacion",texto_limpio_base)
        habilidades(frameworks,"frameworks",texto_limpio_base)
        habilidades(servicios_nube,"nube",texto_limpio_base)
        habilidades(cargos,"cargos",documento_txt)
        habilidades(idiomas,"idiomas",texto_limpio_base)
        habilidades(estudios,"estudios",texto_limpio_base)
        
        json_hv["porcentaje_coincidencias"] = str(coincidencias(parametros, json_hv))
        lista_json.append(json_hv)  
        
    return lista_json

In [17]:
with open('/content/jsonprueba.json') as f:
    json_entrada = json.load(f)

In [18]:
json_entrada

{'bases_de_datos': ['Oracle', 'sql'],
 'cargos': ['Arquitecto de soluciones'],
 'estudios': ['Abbyy Advanced Training'],
 'experiencia': ['4'],
 'frameworks': ['react'],
 'idiomas': ['Ingles'],
 'lenguajes_de_programacion': ['Java'],
 'nube': ['AWS'],
 'sueldo': ['7'],
 'tarjeta_profesional': ['si']}

In [19]:
parsing_hv(json_entrada)

FileNotFoundError: ignored